In [1]:
import pandas as pd
import re
import os
import collections
import nltk
import urllib
import bs4 as bs
import re
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import numpy

from numpy import dot
from numpy.linalg import norm

In [2]:
list_of_files = os.listdir('../top1000_complete')

In [3]:
#Leitura dos artigos
list_of_files2= []
for i in list_of_files:
    nome = '../top1000_complete/' + i + '/' + 'Documents_xml/' +os.listdir(f'../top1000_complete/{i}/Documents_xml')[0]
    list_of_files2.append(nome)
    
    
lines = []
for file in list_of_files2:
    f= open(file,'r')
    lines.append(f.readlines())
    f.close()

In [4]:
#Leitura do padrão ouro
list_of_files3= []
for i in list_of_files:
    nome = '../top1000_complete/' + i + '/' + 'summary/' +os.listdir(f'../top1000_complete/{i}/summary')[0]
    list_of_files3.append(nome)
    
    
lines3 = []
for file in list_of_files3:
    f= open(file,'r',encoding='utf-8')
    lines3.append(f.readlines())
    f.close()

In [5]:
paper1= lines[4]

In [6]:
paper1

['<PAPER>\n',
 '  <S sid="0">A Maximum-Entropy-Inspired Parser</S>\n',
 '  <ABSTRACT>\n',
 '    <S sid="1" ssid="1">We present a new parser for parsing down to Penn tree-bank style parse trees that achieves 90.1% average precision/recall for sentences of 40 and less, and for of length 100 and less when trained and tested on the previously established [5,9,10,15,17] &amp;quot;standard&amp;quot; sections of the Wall Street Journal treebank.</S>\n',
 '    <S sid="2" ssid="2">This represents a 13% decrease in error rate over the best single-parser results on this corpus [9].</S>\n',
 '    <S sid="3" ssid="3">The major technical innovation is the use of a &amp;quot;maximum-entropy-inspired&amp;quot; model for conditioning and smoothing that let us successfully to test and combine many different conditioning events.</S>\n',
 '    <S sid="4" ssid="4">We also present some partial results showing the effects of different conditioning information, including a surprising 2% improvement due to gue

In [7]:
summary1= lines3[4]

In [8]:
summary1

['A Maximum-Entropy-Inspired Parser\n',
 'We present a new parser for parsing down to Penn tree-bank style parse trees that achieves 90.1% average precision/recall for sentences of length 40 and less, and 89.5% for sentences of length 100 and less when trained and tested on the previously established [5,9,10,15,17] "standard" sections of the Wall Street Journal tree- bank.\n',
 'This represents a 13% decrease in error rate over the best single-parser results on this corpus [9].\n',
 'The major technical innovation is the use of a "maximum-entropy-inspired" model for conditioning and smoothing that let us successfully to test and combine many different conditioning events.\n',
 "We also present some partial results showing the effects of different conditioning information, including a surprising 2% improvement due to guessing the lexical head's pre-terminal before guessing the lexical head.\n",
 'As an alternative to hard coded heuristics, we proposed to recover the Penn functional tags

lista3=[]
lines2=''
for i in range(len(paper1)):
    for j in paper1[i]:
        lines2= lines2+j
    lista3.append(lines2)

In [9]:
texto=''
for i in paper1:
    texto = texto + str(i)

In [10]:
def preprocessa_intro(corpus: str):
    regIter = re.finditer('(<SECTION.*Introduction.*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = re.sub(',','', txt)
        txt = re.sub(';','', txt)
        txt = re.sub(':','', txt)
        txt = re.sub('[ ]+',' ', txt)
        txt = re.sub(r'\([^)]*\)', "",txt)
        txt = re.sub(r'\<(.*?)\>',"",txt)
        txt = re.sub(r'small\&',"",txt)
        txt = re.sub(r'tagger&amp',"",txt)
        txt = re.sub(r';quot',"",txt)
        txt = re.sub(r'\&amp',"",txt)
        txt = re.sub(' +', ' ', txt)
        txt = txt.lower()
        #txt = '<sos> ' + txt.strip() + ' <eos>'
        txt = txt.strip()
        txt = re.sub(' . ', '.', txt).strip()
        #txt = '<sos> ' + txt.strip() + ' <eos>'
        return txt
    return txt

def preprocessa_abstract(corpus: str):
    regIter = re.finditer("(<ABSTRACT>)(.+?)(</ABSTRACT>)", corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = re.sub(',','', txt)
        txt = re.sub(';',' ;', txt)
        txt = re.sub('[ ]+',' ', txt)
        txt = re.sub(r'\([^)]*\)', "",txt)
        txt = re.sub(r'\<(.*?)\>',"",txt)
        txt = txt.lower()
        txt = re.sub(' . ', '.', txt).strip()
        #txt = '<sos> ' + txt.strip() + ' <eos>'
        lista.append(txt)
    return txt


def preprocessa_conclusion(corpus: str):
    regIter = re.finditer('(<SECTION.*Conclusion.*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = re.sub(',','', txt)
        txt = re.sub(';','', txt)
        txt = re.sub(':','', txt)
        txt = re.sub('[ ]+',' ', txt)
        txt = re.sub(r'\([^)]*\)', "",txt)
        txt = re.sub(r'\<(.*?)\>',"",txt)
        txt = re.sub(r'small\&',"",txt)
        txt = re.sub(r'tagger&amp',"",txt)
        txt = re.sub(r';quot',"",txt)
        txt = re.sub(r'\&amp',"",txt)
        txt = re.sub(' +', ' ', txt)
        txt = txt.lower()
        txt = re.sub(' . ', '.', txt).strip()
        #txt = '<sos> ' + txt.strip() + ' <eos>'
        txt = txt.strip()
        #txt = '<sos> ' + txt.strip() + ' <eos>'
        return txt
    return txt

In [11]:
def remove_values_from_list(the_list, val):
    return [value for value in the_list if value != val]

In [12]:
abstract   = preprocessa_abstract(texto)
intro      = preprocessa_intro(texto)
conclusion = preprocessa_conclusion(texto)

In [13]:
token_abstract   = preprocessa_abstract(texto).split()
token_intro      = preprocessa_intro(texto).split()
token_conclusion = preprocessa_conclusion(texto).split()

In [14]:
tokens = [token_abstract+token_intro+token_conclusion]

In [15]:
model = Word2Vec(tokens, min_count=1)

In [16]:
#model.wv.key_to_index

frases_abstract = abstract.split('.')
palavras_frases=[]
for i in frases_abstract:
    palavras_frases.append(i.split(' '))
palavras_frases

In [17]:
def funcao_frase_palavra(abstract):
    """input: lista de frases
        output: lista de palavras de cada frase"""
    frases_abstract = abstract.split('.')
    palavras_frases=[]
    for i in frases_abstract:
        palavras_frases.append(i.split(' '))
    try:
        palavras_frases.remove('')
    except:
        return palavras_frases
    return palavras_frases

In [18]:
def vetor_frases(palavras_frases):
    """input: lista de palavras de cada frase
        output: lista de vetores de cada frase"""
    x=[]
    for i in palavras_frases:
        soma=0
        for j in i:
            try:
                soma = soma +model.wv[j]
            except:
                soma=soma+0
        x.append(soma)
    return x

In [19]:
def funcao_sim_coseno(a,b):
    valor_coseno = (numpy.dot(a, b))/(numpy.linalg.norm(a)* numpy.linalg.norm(b))
    if valor_coseno  is None:
        return 0
    else:
        return valor_coseno

In [20]:
frases_abstract   = abstract.split('.')
palavras_abstract = funcao_frase_palavra(abstract)
palavras_abstract = remove_values_from_list(palavras_abstract, [''])
vetor_abstract    = vetor_frases(palavras_abstract)

In [21]:
frases_intro   = intro.split('.')
palavras_intro = funcao_frase_palavra(intro)
palavras_intro = remove_values_from_list(palavras_intro, [''])
vetor_intro    = vetor_frases(palavras_intro)

In [22]:
frases_conclusion   = conclusion.split('.')
palavras_conclusion = funcao_frase_palavra(conclusion)
palavras_conclusion = remove_values_from_list(palavras_conclusion, [''])
vetor_conclusion    = vetor_frases(palavras_conclusion)

In [23]:
def funcao_sim_coseno(a,b):
    if ((numpy.linalg.norm(a)==0) |(numpy.linalg.norm(b)==0)):
        return 0
    else:
        valor_coseno = (numpy.dot(a, b))/(numpy.linalg.norm(a)* numpy.linalg.norm(b))
        return valor_coseno

In [24]:
def dicionario_similaridade(vetor_a,vetor_b):
    """Input: vetor correspondente às frases das seções a e b
    Output: 
    """
    dicionario={}
    for i in range(len(vetor_a)):
        for j in range(len(vetor_b)):
            dicionario[i,j]= funcao_sim_coseno(vetor_a[i],vetor_b[j])
    return dicionario

In [25]:
dicionario = dicionario_similaridade(vetor_abstract, vetor_intro)

In [26]:
top_3_indices = sorted(dicionario, key=dicionario.get, reverse=True)[:3]
top_3_indices

[(4, 4), (0, 0), (5, 5)]

In [27]:
for i in top_3_indices:
    print(dicionario[i])

1.0000001
1.0
1.0


In [28]:
x=[]
for i in range(len(top_3_indices)):
    x.append(top_3_indices[i][0])
indice_abstract = list(set(x))

In [29]:
indice_abstract

[0, 4, 5]

In [30]:
vetor_frases_objetivo_1=[]
for i in range(len(indice_abstract)):
    vetor_frases_objetivo_1.append(vetor_intro[indice_abstract[i]])
vetor_frases_objetivo_1

[array([-0.02728748,  0.01632557,  0.00221955,  0.00257971,  0.00192509,
        -0.04115362,  0.02328084,  0.06711588, -0.02734399, -0.01915182,
        -0.01015315, -0.02085561, -0.02201623,  0.02554366,  0.00402589,
        -0.02290612, -0.00049584, -0.01995485, -0.01418897, -0.05465411,
         0.01697383, -0.00563399,  0.01977037,  0.00327611,  0.00041953,
         0.01290847, -0.02054577, -0.01113396, -0.03020561,  0.00741656,
         0.02760772,  0.00322055,  0.01196965, -0.02077815, -0.00015326,
         0.02408354,  0.01246118, -0.01871097, -0.01979257, -0.0458454 ,
         0.01620316, -0.01887285, -0.02552225,  0.01766346,  0.02402374,
        -0.02519953, -0.01828353, -0.0050073 ,  0.00847415, -0.00221595,
         0.01309613, -0.02439694, -0.00178752, -0.01187825, -0.01294066,
         0.01023848,  0.00355543, -0.01672422, -0.01753071, -0.00675693,
         0.0217683 ,  0.0024905 , -0.00699062, -0.01117368, -0.03220685,
         0.02611551,  0.01544785,  0.03255321, -0.0

In [31]:
dicionario = dicionario_similaridade(vetor_abstract, vetor_conclusion)

In [32]:
top_3_indices = sorted(dicionario, key=dicionario.get, reverse=True)[:3]
top_3_indices

[(9, 15), (6, 10), (5, 4)]

In [33]:
for i in top_3_indices:
    print(dicionario[i])

0.99118465
0.98977053
0.98917323


In [34]:
x=[]
for i in range(len(top_3_indices)):
    x.append(top_3_indices[i][0])
indice_abstract = list(set(x))

In [35]:
vetor_frases_objetivo_1=[]
for i in range(len(indice_abstract)):
    vetor_frases_objetivo_1.append(vetor_intro[indice_abstract[i]])
vetor_frases_objetivo_1

[array([-0.07823716,  0.09580041,  0.02683538,  0.01079913, -0.00441369,
        -0.14804745,  0.07787856,  0.19914927, -0.08087862, -0.06989818,
        -0.02900805, -0.09462047,  0.01060875,  0.05417459,  0.04872794,
        -0.05723944,  0.03303472, -0.05731393, -0.05514907, -0.18451026,
         0.07052866,  0.04800124,  0.06093913, -0.03499268,  0.01221963,
         0.0123376 , -0.03587793, -0.04285754, -0.08860978, -0.00841041,
         0.08553695, -0.00142937,  0.05162355, -0.06638402, -0.04351277,
         0.11564371,  0.02396262, -0.06110545, -0.0748954 , -0.15986326,
         0.02041036, -0.03792544, -0.03317028,  0.00383806,  0.0480101 ,
        -0.04081042, -0.05726702, -0.0023928 ,  0.06197588,  0.06988463,
         0.0331066 , -0.05647109, -0.02841944, -0.02355552, -0.00790779,
         0.00119575,  0.08153088, -0.0216416 , -0.07464842,  0.01564708,
         0.02458653,  0.00827728,  0.00208555, -0.03690244, -0.07212068,
         0.0774187 ,  0.00234101,  0.06577178, -0.1

In [36]:
for i in indice_abstract:
    print(frases_abstract[i])

 we also present some partial results showing the effects of different conditioning information including
13% decrease in error rate over the best single-parser results on this corpus [9]
 the major technical innovation is the use of


In [37]:
summary1

['A Maximum-Entropy-Inspired Parser\n',
 'We present a new parser for parsing down to Penn tree-bank style parse trees that achieves 90.1% average precision/recall for sentences of length 40 and less, and 89.5% for sentences of length 100 and less when trained and tested on the previously established [5,9,10,15,17] "standard" sections of the Wall Street Journal tree- bank.\n',
 'This represents a 13% decrease in error rate over the best single-parser results on this corpus [9].\n',
 'The major technical innovation is the use of a "maximum-entropy-inspired" model for conditioning and smoothing that let us successfully to test and combine many different conditioning events.\n',
 "We also present some partial results showing the effects of different conditioning information, including a surprising 2% improvement due to guessing the lexical head's pre-terminal before guessing the lexical head.\n",
 'As an alternative to hard coded heuristics, we proposed to recover the Penn functional tags

In [38]:
abstract

"we present.new parser for parsing down to penn tree-bank style parse trees that achieves 90.1% average precision/recall for sentences of 40 and less and for of length 100 and less when trained and tested on the previously established [59101517] &amp ;quot ;standard&amp ;quot.sections of the wall street journal treebank. this represents.13% decrease in error rate over the best single-parser results on this corpus [9]. the major technical innovation is the use of.&amp ;quot ;maximum-entropy-inspired&amp ;quot.model for conditioning and smoothing that let us successfully to test and combine many different conditioning events. we also present some partial results showing the effects of different conditioning information including.surprising 2% improvement due to guessing the lexical head's pre-terminal before guessing the lexical head."